In [2]:
%autosave 0

Autosave disabled


In [ ]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [ ]:
!pip install keras-image-helper

In [6]:
import grpc

import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [7]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [10]:
from keras_image_helper import create_preprocessor

In [11]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [9]:
url ='http://bit.ly/mlbookcamp-pants'

In [16]:
X = preprocessor.from_url(url)

In [17]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape= data.shape)

In [21]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_2'].CopyFrom(np_to_protobuf(X))

In [23]:
pb_response = stub.Predict(pb_request,timeout=20.0)

In [26]:
preds = pb_response.outputs['dense_1'].float_val

In [27]:
classes = [
    'dress',
     'hat',
     'longsleeve',
     'outwear',
     'pants',
     'shirt',
     'shoes',
     'shorts',
     'skirt',
     't-shirt',]

In [28]:
dict(zip(classes, preds))

{'dress': -3.324326276779175,
 'hat': -6.603317737579346,
 'longsleeve': -2.716560125350952,
 'outwear': -3.733776807785034,
 'pants': 10.822230339050293,
 'shirt': -2.882204294204712,
 'shoes': -5.1022138595581055,
 'shorts': 2.5007410049438477,
 'skirt': -5.319211483001709,
 't-shirt': -6.669097423553467}